# Crawling - Scrapy para obtener datos de noticias locales de Madrid

La dirección de noticias locales de Madrid que voy a utilizar es: https://www.eldistrito.es/distritos/
Esta dirección me viene muy bien porque tiene las noticas separadas por distritos que posteriormente podré asociar a barrios en los que están las viviendas de Airbnb

Con este Scrapy consigo un csv con los siguientes datos: fecha de la noticia, distrito y titular de la noticia.

En principio para no "machacar" mucho la página solo estoy accediendo a 4 páginas de noticias que son un total de 32 noticas

In [0]:
!pip install scrapy

In [0]:
import scrapy
import json

class NoticiasItem(scrapy.Item):
    distrito = scrapy.Field()
    titular = scrapy.Field()
    fecha = scrapy.Field()

class DistritoSpider(scrapy.Spider):
    name = 'distritospider'
    # URL Inicial
    start_urls = ['https://www.eldistrito.es/distritos/']
    
    def parse(self, response):
        
        for noticia in response.css('div.noticiaportada'):
            item = NoticiasItem ()
            #Me voy a quedar con el distrito y el titular de la noticia
            item['distrito'] = noticia.css('div.ruta2_sin a::text').extract()[-1]
            item ['titular'] = noticia.css('h1.titular a b::text').extract_first() 
            
            #Me interesa también la fecha de la noticia, para eso hay que entrar en el link de la noticia
            pagina_noticia = noticia.css ('h1.titular a::attr(href)').extract_first()
            yield response.follow(pagina_noticia, meta ={"item":item}, callback=self.parse_noticia)
            
            
       
        for next_page in response.css ('td.ant_sig a'):
            #Hay 2 href, el primero va hacia la página anterior y el segundo hacia la siguiente por eso me quedo
            #con el último [-1]
            href = response.css ('td.ant_sig a::attr(href)').extract()[-1]
            if href is not None:
                #Para no hacer mucho daño me quedo solo con 3 páginas de noticias, van de 8 en 8
                #Para un proyecto final podemos quitar esta condición y traernos todo
                if int(href.split('/')[-1]) > 24:
                    return
            
                yield response.follow(href, self.parse)
                
    def parse_noticia (self, response): 
        #Dentro de la noticia para obtener la fecha
        item = response.meta ["item"]
        item['fecha'] = response.css ('span.fecha::text').extract_first()
        f=item['fecha']
        d=item['distrito']
        t=item['titular']
        print(f"{f};{d}; {t}", file=filep)

In [0]:
from scrapy.crawler import CrawlerProcess


process = CrawlerProcess({
    'USER_AGENT': 'Crawler test - formacion bootcamp bigdata'
})

filep = open('datanoticias.csv', 'w')
print("{};{};{}".format("fecha", "distrito", "titular"), file=filep)
process.crawl(DistritoSpider)
process.start()
filep.close()


In [0]:
from google.colab import files
files.download('datanoticias.csv')

**Con el siguiente código podemos cargar directamente el fichero en el Google Storage**

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')

In [0]:
from googleapiclient.http import MediaFileUpload

media = MediaFileUpload('datanoticias.csv', 
                        mimetype='text/plain',
                        resumable=True)

request = gcs_service.objects().insert(bucket='dataproc-fe7b85fd-43dc-4de0-a520-a824fcd432de-europe-west1', 
                                       name='datanoticias.csv',
                                       media_body=media)

response = None
while response is None:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, response = request.next_chunk()

print('Upload complete')